<a href="https://colab.research.google.com/github/liliya111darky/logical-ai/blob/notebooks/03_logic_core.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Устанавливаем нужные библиотеки
!pip install sympy transformers

# Импортируем всё необходимое
from sympy import sympify, simplify
from transformers import pipeline

# NLP-модель для анализа тональности
classifier = pipeline("sentiment-analysis")

# Функция для проверки математических выражений
def math_check(statement):
    try:
        # Заменяем текстовые операторы на математические
        statement = statement.replace("и", "&").replace("или", "|").replace("не", "~")
        if any(op in statement for op in ["=", ">", "<"]):
            if "=" in statement:
                left, right = statement.split("=")
                return simplify(sympify(left) - sympify(right)) == 0
            else:
                # Для неравенств просто проверим через eval SymPy
                return bool(sympify(statement))
        return None  # Не математика
    except:
        return None  # Не удалось интерпретировать как математику

# Главная функция анализа
def logic_core(text):
    math_result = math_check(text)
    if math_result is True:
        sentiment = classifier(text)[0]
        print(f"✅ Математическая проверка пройдена.")
        print(f"   Настроение: {sentiment['label']} ({sentiment['score']:.2%})")
    elif math_result is False:
        print(f"❌ Математическая проверка не пройдена. Анализ не выполняется.")
    else:
        sentiment = classifier(text)[0]
        print(f"ℹ️ Это не математика.")
        print(f"   Настроение: {sentiment['label']} ({sentiment['score']:.2%})")

# Примеры
logic_core("2+2=4")
logic_core("5 > 3 и 3 > 1")
logic_core("2+2=5 грустно")
logic_core("Сегодня отличный день!")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cpu


✅ Математическая проверка пройдена.
   Настроение: POSITIVE (95.96%)
✅ Математическая проверка пройдена.
   Настроение: POSITIVE (62.20%)
ℹ️ Это не математика.
   Настроение: POSITIVE (74.83%)
ℹ️ Это не математика.
   Настроение: POSITIVE (96.19%)


In [1]:
# Установка зависимостей (выполняется один раз)
!pip install z3-solver sympy transformers --quiet

# Импорты
import re
from z3 import Real, Solver, sat, unsat, Not
from transformers import pipeline

# Инициализация NLP-модели (как до этого)
classifier = pipeline("sentiment-analysis")

# ------- Вспомогательные функции для разбора и преобразования --------

def normalize_text(s: str) -> str:
    """Небольшая нормализация: убираем лишние пробелы."""
    return s.strip()

def extract_conditional(text: str):
    """
    Ищем конструкции вида 'Если ... то ...' (регистронезависимо).
    Возвращаем (antecedent, consequent) или (None, None) если не conditional.
    """
    t = text.strip()
    # ищем 'если' ... 'то' (ленивый режим)
    m = re.search(r'(?i)\bесли\b(.+?)\bто\b(.+)', t, flags=re.IGNORECASE)
    if m:
        a = m.group(1).strip()
        c = m.group(2).strip()
        return a, c
    # попутно поддержим стрелочные записи A => B или A -> B
    m = re.search(r'(.+?)->(.+)', t)
    if m:
        return m.group(1).strip(), m.group(2).strip()
    m = re.search(r'(.+?)=>\s*(.+)', t)
    if m:
        return m.group(1).strip(), m.group(2).strip()
    return None, None

def prepare_expr(expr: str) -> str:
    """
    Подготовка выражения: переводим русские логические слова в символы,
    меняем '=' на '==' (ровное) корректно, '^'->'**' и т.д.
    Возвращаем строку, пригодную для eval с Z3-переменными (используем &,|,~).
    """
    s = expr
    # границы слов, чтобы не ломать части слов
    s = re.sub(r'(?i)\bи\b', '&', s)      # 'и' -> &
    s = re.sub(r'(?i)\bили\b', '|', s)    # 'или' -> |
    s = re.sub(r'(?i)\bне\b', '~', s)     # 'не' -> ~
    # заменить одиночный '=' на '==', не трогая '>=', '<=', '!=' и '=='
    s = re.sub(r'(?<![<>=!])=(?!=)', '==', s)
    # степень: ^ -> **
    s = s.replace('^', '**')
    # убрать лишние запятые/точки, если мешают
    return s

def collect_vars(expr: str, var_dict: dict):
    """Находит идентификаторы (латиница/кириллица/подчёркивания) и создаёт z3.Real."""
    tokens = re.findall(r'\b[A-Za-zА-Яа-яёЁ_]\w*\b', expr)
    for t in tokens:
        # пропускаем логические символы/ключевые слова
        if t.lower() in ('and','or','not','true','false'):
            continue
        if re.fullmatch(r'\d+', t):  # чисто число
            continue
        if t not in var_dict:
            var_dict[t] = Real(t)
    return var_dict

# ------- Основные проверки --------

def check_implication(antecedent_str: str, consequent_str: str):
    """
    Проверяет, является ли (antecedent) => (consequent) логически истинным (для всех значений переменных).
    Метод: проверяем выполнимость (antecedent) & not(consequent). Если UNSAT -> импликация верна.
    Возвращает (is_valid, counterexample_model_or_None)
    """
    var_dict = {}
    a_p = prepare_expr(antecedent_str)
    c_p = prepare_expr(consequent_str)
    # собираем переменные
    collect_vars(a_p, var_dict)
    collect_vars(c_p, var_dict)
    try:
        A = eval(a_p, {}, var_dict)
        C = eval(c_p, {}, var_dict)
    except Exception as e:
        return None, f"Не удалось распознать выражения: {e}"
    s = Solver()
    s.push()
    s.add(A)
    s.add(Not(C))
    res = s.check()
    if res == unsat:
        s.pop()
        return True, None
    elif res == sat:
        model = s.model()
        # Сформируем читаемый контрпример
        example = {str(d): model[d] for d in model}
        s.pop()
        return False, example
    else:
        s.pop()
        return None, "Неоднозначный результат (unknown)"

def check_expression(expr_str: str):
    """
    Для одиночных утверждений:
    - Если в выражении нет переменных — вычисляем напрямую (True/False).
    - Если есть переменные — проверяем выполнимость (существует ли модель, делающая выражение истинным).
    Возвращает (result_bool_or_None, model_or_message)
    """
    var_dict = {}
    p = prepare_expr(expr_str)
    collect_vars(p, var_dict)
    try:
        E = eval(p, {}, var_dict)
    except Exception as e:
        return None, f"Не удалось распознать выражение: {e}"
    # Если выражение — чистый Python bool (нет переменных), просто возвращаем
    if isinstance(E, bool):
        return E, None
    # Иначе — это Z3 BoolRef: проверяем выполнимость (есть ли модель, делающая выражение True)
    s = Solver()
    s.push()
    s.add(E)
    res = s.check()
    if res == sat:
        model = s.model()
        example = {str(d): model[d] for d in model}
        s.pop()
        return True, example
    elif res == unsat:
        s.pop()
        return False, None
    else:
        s.pop()
        return None, "unknown"

# ------- Объединённая функция "ядра" --------

def logic_core_with_conditions(text: str):
    """
    Главная функция:
    - если текст содержит 'Если ... то ...' — делаем проверку импликации
    - иначе, если текст содержит знаки сравнения/равенства — проверяем выражение
    - иначе — это не-математика: просто отправляем в NLP (тональность)
    """
    text = normalize_text(text)
    antecedent, consequent = extract_conditional(text)
    if antecedent:
        valid, info = check_implication(antecedent, consequent)
        if valid is True:
            sentiment = classifier(text)[0]
            return {
                "result": "valid_implication",
                "message": "✅ Импликация доказана (для всех значений переменных).",
                "antecedent": antecedent,
                "consequent": consequent,
                "sentiment": sentiment
            }
        elif valid is False:
            return {
                "result": "invalid_implication",
                "message": "❌ Импликация не верна — найден контрпример.",
                "antecedent": antecedent,
                "consequent": consequent,
                "counterexample": info
            }
        else:
            return {"result": "error", "message": info}
    else:
        # Проверяем, есть ли в тексте математические операторы
        if re.search(r'[=<>]', text):
            ok, info = check_expression(text)
            if ok is True:
                # математическое выражение выполнимо/истинно
                sentiment = classifier(text)[0]
                return {
                    "result": "math_true",
                    "message": "✅ Математическая проверка пройдена (возможно, найдена модель).",
                    "expression": text,
                    "model": info,
                    "sentiment": sentiment
                }
            elif ok is False:
                return {"result": "math_false", "message": "❌ Математическая проверка не пройдена.", "expression": text}
            else:
                return {"result": "error", "message": info}
        else:
            # Нет математики — обычный NLP-анализ
            sentiment = classifier(text)[0]
            return {
                "result": "nlp_only",
                "message": "ℹ️ Это не математическое выражение.",
                "sentiment": sentiment
            }

# --------- Примеры использования ----------
tests = [
    "Если A > B и B > C, то A > C",
    "2+2=4",
    "2+2=5",
    "x+2=5",
    "x > x",
    "Сегодня чудесный день!"
]

for t in tests:
    print("----")
    print("Ввод:", t)
    out = logic_core_with_conditions(t)
    print("Вывод:", out)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.5/29.5 MB 23.0 MB/s eta 0:00:00


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cpu


----
Ввод: Если A > B и B > C, то A > C
Вывод: {'result': 'error', 'message': "Не удалось распознать выражения: unsupported operand type(s) for &: 'ArithRef' and 'ArithRef'"}
----
Ввод: 2+2=4
Вывод: {'result': 'math_true', 'message': '✅ Математическая проверка пройдена (возможно, найдена модель).', 'expression': '2+2=4', 'model': None, 'sentiment': {'label': 'POSITIVE', 'score': 0.9596148729324341}}
----
Ввод: 2+2=5
Вывод: {'result': 'math_false', 'message': '❌ Математическая проверка не пройдена.', 'expression': '2+2=5'}
----
Ввод: x+2=5
Вывод: {'result': 'math_true', 'message': '✅ Математическая проверка пройдена (возможно, найдена модель).', 'expression': 'x+2=5', 'model': {'x': 3}, 'sentiment': {'label': 'POSITIVE', 'score': 0.9468357563018799}}
----
Ввод: x > x
Вывод: {'result': 'math_false', 'message': '❌ Математическая проверка не пройдена.', 'expression': 'x > x'}
----
Ввод: Сегодня чудесный день!
Вывод: {'result': 'nlp_only', 'message': 'ℹ️ Это не математическое выражение.', '